<a href="https://colab.research.google.com/github/Eman-Adly/Eman/blob/main/gpt2_f2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [2]:
file_path = "/content/updated_book_recommendation_data.csv"
df = pd.read_csv(file_path)

In [3]:
df

,student_id,book_id,author,title,rating,department,genre,year_of_study,published_year,num_pages,reading_frequency
0,8538,1544,Kyle Alvarado,Maintain interest manage evidence,2,Mathematics,Fantasy,4,2000,165,High
1,6131,2685,Chloe Medina,Reality free,2,Medicine,Science Fiction,2,1963,589,Low
2,1734,7678,William Edwards,Prepare management kitchen cut,5,Engineering,Educational,1,2007,751,High
3,2108,9053,Todd Steele,Modern moment provide receive image,2,Business,Mystery,4,1989,399,Low
4,3772,1641,Lisa Joseph,Traditional establish couple usually,5,History,Science Fiction,3,2018,528,Medium
...,...,...,...,...,...,...,...,...,...,...,...
1995,3844,8390,James Cherry,Hospital then,2,Physics,Non-Fiction,2,1967,912,Medium
1996,9831,3759,Anthony Haley,Laugh stuff there home sense,5,History,Thriller,1,1997,789,Medium
1997,1259,1047,Maria Adams,Step let entire reality course,3,Computer Science,Fantasy,2,1970,614,Low
1998,8720,8976,Susan Shepherd,Local become,2,Philosophy,Historical,2,1974,159,Low


In [4]:
df.isnull().sum()

,0
student_id,0
book_id,0
author,0
title,0
rating,0
department,0
genre,0
year_of_study,0
published_year,0
num_pages,0


In [6]:
df.duplicated().sum()

0

In [7]:
df.dropna(inplace=True)

In [15]:
interaction_matrix = df.pivot(index="student_id", columns="title", values="rating").fillna(0)

In [16]:

interaction_sparse = csr_matrix(interaction_matrix)
student_similarity = cosine_similarity(interaction_sparse)

student_sim_df = pd.DataFrame(student_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)


def recommend_books_collaborative(student_id, student_department, student_year, top_n=5):

    if student_id not in interaction_matrix.index:
        print(" New Student ")
        top_books = df[df["department"] == student_department].sort_values(by="rating", ascending=False).head(top_n)
        return top_books[["author", "title", "genre", "published_year", "num_pages", "rating"]]


    similar_students = student_sim_df[student_id].sort_values(ascending=False)[1:6]

    similar_students_books = interaction_matrix.loc[similar_students.index]

    book_recommendations = similar_students_books.mean(axis=0)


    books_read_by_student = interaction_matrix.loc[student_id]
    unread_books = book_recommendations[books_read_by_student == 0]

    top_recommended_books = unread_books.sort_values(ascending=False).head(top_n)

    final_recommendations = df[df["title"].isin(top_recommended_books.index)][["author", "title", "genre", "published_year", "num_pages", "rating"]].drop_duplicates()

    return final_recommendations




In [9]:
# enter the (student_id	, department , year_of_study )    old student

print(recommend_books_collaborative(8538, "Mathematics", 4))

              author                        title            genre  \
578   Kimberly Brown                Laugh surface          Fantasy   
733    Michelle Ross  Two available continue edge  Science Fiction   
1005   Tracy Collins       President effect while         Thriller   
1344     Lisa Rogers    Her message international        Self-Help   
1563      Isaac Hahn                       Can by      Non-Fiction   

      published_year  num_pages  rating  
578             1988        685       5  
733             1980        159       4  
1005            2015        367       3  
1344            1979        476       3  
1563            1961        656       4  


In [12]:
# # enter the (student_id	, department , year_of_study )    new student
print(recommend_books_collaborative(1, "Mathematics", 4))

 New Student 
                author                        title        genre  \
1991    Brittany Young                   Issue able    Biography   
348       William Cole   Development parent boy bar  Non-Fiction   
476      Brenda Howell         Writer suddenly lead      Fantasy   
1448  Nicholas Wilkins  Individual father pull step  Educational   
569       Donna Levine                  Water trade      Fantasy   

      published_year  num_pages  rating  
1991            1974        731       5  
348             1995        746       5  
476             1951        112       5  
1448            2007        604       5  
569             2017        292       5  


# **Collaborative Filtering**

In [18]:
#  New Student
print(recommend_books_collaborative(3, "Engineering", 1))

 New Student 
                author                                   title  \
2      William Edwards          Prepare management kitchen cut   
1479        John Clark                        Forget operation   
1436         Dana Case           Likely people public war week   
1420  Raymond Delacruz                         Left weight gun   
1346       Lisa Briggs  Certainly general ready prove Democrat   

                genre  published_year  num_pages  rating  
2         Educational            2007        751       5  
1479  Science Fiction            2024        408       5  
1436          Fantasy            1954        406       5  
1420          Fantasy            1976        879       5  
1346          Mystery            1958        571       5  


In [14]:
# old student
print(recommend_books_collaborative(1734, "Engineering", 1))

               author                        title            genre  \
578    Kimberly Brown                Laugh surface          Fantasy   
733     Michelle Ross  Two available continue edge  Science Fiction   
1311  Heather Collins                Region effect      Educational   
1344      Lisa Rogers    Her message international        Self-Help   
1563       Isaac Hahn                       Can by      Non-Fiction   

      published_year  num_pages  rating  
578             1988        685       5  
733             1980        159       4  
1311            1975        703       4  
1344            1979        476       3  
1563            1961        656       4  
